In [1]:
#load packages
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import json




/work/MarkusLundsfrydJensen#1865/miniconda3/envs/blame_bert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm
import numpy as np

class BlameDetectorDa(object):

    def __init__(self, model_path, max_length=512, batch_size=32):

        self.model_path = model_path
        self.max_length = max_length
        self.batch_size = batch_size

        self.model_initialization()

        return

    def model_initialization(self):
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_path,
            device_map='auto'
        )

        self.model.eval()

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model.to(self.device)

        print(f"Model loaded successfully on {self.device}")

        return

    def predict(self, text):
        """Make a prediction on a single text input."""
        # Tokenize input
        inputs = self.tokenizer(
            text,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
       
        # Move inputs to device
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        # Make prediction
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()
        
        return predicted_class, confidence, probabilities[0].cpu().numpy()

    def predict_batch(self, texts):
        """Make predictions on a batch of texts."""
        # Tokenize all texts in the batch
        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Move inputs to device
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        # Make predictions
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_classes = torch.argmax(probabilities, dim=1).cpu().numpy()
            confidences = probabilities[range(len(predicted_classes)), predicted_classes].cpu().numpy()
        
        return predicted_classes, confidences, probabilities.cpu().numpy()

    def run_prediction(self, text):
        """Single text prediction (backward compatibility)."""
        predicted_class, confidence, probs = self.predict(text)
        return predicted_class, confidence

    def run_batch_prediction(self, texts, show_progress=True):
        """
        Run predictions on a list of texts with batching and progress bar.
        
        Args:
            texts: List of text strings to predict on
            show_progress: Whether to show progress bar (default: True)
            
        Returns:
            predicted_classes: numpy array of predicted class indices
            confidences: numpy array of confidence scores
        """
        all_predictions = []
        all_confidences = []
        
        # Create batches
        num_batches = (len(texts) + self.batch_size - 1) // self.batch_size
        
        # Setup progress bar
        if show_progress:
            pbar = tqdm(total=len(texts), desc="Processing texts", unit="text")
        
        # Process in batches
        for i in range(0, len(texts), self.batch_size):
            batch_texts = texts[i:i + self.batch_size]
            
            # Get predictions for this batch
            predicted_classes, confidences, _ = self.predict_batch(batch_texts)
            
            all_predictions.extend(predicted_classes)
            all_confidences.extend(confidences)
            
            # Update progress bar
            if show_progress:
                pbar.update(len(batch_texts))
        
        if show_progress:
            pbar.close()
        
        return np.array(all_predictions), np.array(all_confidences)


    def predict_from_json(self, json_path, text_key = 'text', output_key='prediction', 
                          confidence_key='confidence', show_progress=True):
        """
        Load JSON, predict, and return results with predictions added.
        
        Args:
            json_path: Path to JSON file
            text_key: Key in JSON objects containing the text to classify
            output_key: Key name for storing predictions (default: 'prediction')
            confidence_key: Key name for storing confidence scores (default: 'confidence')
            show_progress: Whether to show progress bar
            
        Returns:
            List of dictionaries with predictions added
        """
        import json
        
        # Load JSON data
        print(f"Loading data from {json_path}...")
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        print('json file loaded')
        
        # Handle different JSON structures
        if isinstance(data, list):
            items = data
        elif isinstance(data, dict):
            # If it's a dict, try to find the list of items
            # Adjust this based on your JSON structure
            items = list(data.values()) if all(isinstance(v, dict) for v in data.values()) else [data]
        else:
            raise ValueError("Unsupported JSON structure")
        
        print(f"Found {len(items)} items. extracting texts...")
        
        # Extract texts
        texts = [item[text_key] for item in items]

        print('start batch prediction')
        
        # Run predictions
        predictions, confidences = self.run_batch_prediction(texts, show_progress=show_progress)
        
        # Add predictions to original data
        for item, pred, conf in zip(items, predictions, confidences):
            item[output_key] = int(pred)
            item[confidence_key] = float(conf)
        
        return items

    def predict_from_json_to_file(self, input_path, output_path, text_key, 
                                   output_key='prediction', confidence_key='confidence',
                                   show_progress=True):
        """
        Load JSON, predict, and save results to a new file.
        
        Args:
            input_path: Path to input JSON file
            output_path: Path to save output JSON file
            text_key: Key containing text to classify
            output_key: Key name for predictions
            confidence_key: Key name for confidence scores
            show_progress: Whether to show progress bar
        """
        import json
        
        # Get predictions
        results = self.predict_from_json(
            input_path, text_key, output_key, confidence_key, show_progress
        )
        
        # Save to file
        print(f"Saving results to {output_path}...")
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        
        print(f"Done! Processed {len(results)} items")
        
        return results

In [5]:
detector = BlameDetectorDa(
    model_path="Lundsfryd/blameBERT",
    max_length=512,
    batch_size=64
)

Model loaded successfully on cuda


In [6]:

# Method 2: Process and save to file directly
detector.predict_from_json_to_file(
    input_path="/work/MarkusLundsfrydJensen#1865/final_inference_data.json",
    output_path="/work/MarkusLundsfrydJensen#1865/final_final_inference_data.json",
    text_key="text"
)


Loading data from /work/MarkusLundsfrydJensen#1865/final_inference_data.json...


Processing texts:   1%|          | 19328/2191051 [01:03<1:06:37, 543.30text/s]

json file loaded
Found 2191051 items. extracting texts...
start batch prediction


Processing texts: 100%|██████████| 2191051/2191051 [1:03:55<00:00, 571.29text/s]


Saving results to /work/MarkusLundsfrydJensen#1865/final_final_inference_data.json...
Done! Processed 2191051 items


[{'paragraph': '36314',
  'sentence_nr': '0',
  'text': 'Til at besvare spørgsmål i spørgetimen i dag har  statsministeren udpeget fødevareministeren.',
  'speaker': 'Henning Grove',
  'party': None,
  'current_speaker_in_government': False,
  'parties_in_government': ['S', 'RV'],
  'date': '2000-11-01 00:00:00',
  'prediction': 0,
  'confidence': 0.9971687197685242},
 {'paragraph': '36314',
  'sentence_nr': '1',
  'text': 'Der er indtil dette øjeblik tilmeldt fem hovedspørgere  til fødevareministeren til denne spørgetime.',
  'speaker': 'Henning Grove',
  'party': None,
  'current_speaker_in_government': False,
  'parties_in_government': ['S', 'RV'],
  'date': '2000-11-01 00:00:00',
  'prediction': 0,
  'confidence': 0.9994408488273621},
 {'paragraph': '36314',
  'sentence_nr': '2',
  'text': 'Er der flere,  der ønsker at tilmelde sig som hovedspørgere?',
  'speaker': 'Henning Grove',
  'party': None,
  'current_speaker_in_government': False,
  'parties_in_government': ['S', 'RV'],
  

In [8]:
import json
import pandas as pd
json_inf_data = pd.read_json("/work/MarkusLundsfrydJensen#1865/final_final_inference_data.json")

json_inf_data.to_csv("/work/MarkusLundsfrydJensen#1865/inference_data.csv", index=False)

In [13]:
sum(json_inf_data["prediction"])

114383

In [20]:
test_text = "Det skal regeringen bestemt ikke have tak for"

detector.run_prediction(test_text)

(1, 0.8849397897720337)

Jeg oplever det lidt som en dårlig undskyldning.
(0, 0.7059693932533264)

Det er en svær diskussion, jeg rejser, og det er for let bare at løbe hen i et hjørne og sige: Vi skal have flere ressourcer og flere lærere.
(0, 0.9753536581993103)

Ja, ja, ja, siger jeg til det hele.
(0, 0.9524555206298828)

Men det ændrer ikke på, at selv en skole med flere penge og flere lærere har behov for klare spilleregler og respekt for fællesskabet.
(0, 0.9961349964141846)

Og at vi forældre påtager os den opgave, at vores børn opfører sig ordentligt.
(0, 0.7020463347434998)

